# Sort by Key

Algorithm that sorts the elements of a set of files and merges the partial results respecting the order.

### First of all - Create a dataset

This step can be avoided if the dataset already exists.

If not, this code snipped creates a set of files with dictionary on each one generated randomly. Uses pickle.

In [1]:
def datasetGenerator(directory, numFiles, numPairs):
    import random
    import pickle
    import os
    if os.path.exists(directory):
        print("Dataset directory already exists... Removing")
        import shutil
        shutil.rmtree(directory)
    os.makedirs(directory)
    for f in range(numFiles):
        fragment = {}
        while len(fragment) < numPairs:
            fragment[random.random()] = random.randint(0, 1000)
        filename = 'file_' + str(f) + '.data'
        with open(directory + '/' + filename, 'wb') as fd:
            pickle.dump(fragment, fd)
        print('File ' + filename + ' has been created.')

In [2]:
numFiles = 2
numPairs = 10
directoryName = 'mydataset'
datasetGenerator(directoryName, numFiles, numPairs)

Dataset directory already exists... Removing
File file_0.data has been created.
File file_1.data has been created.


In [3]:
# Show the files that have been created
%ls -l $directoryName

total 8
-rw-r--r-- 1 javier users 134 ago 18 12:37 file_0.data
-rw-r--r-- 1 javier users 134 ago 18 12:37 file_1.data


### Algorithm definition

In [4]:
import pycompss.interactive as ipycompss

In [5]:
import os
if 'BINDER_SERVICE_HOST' in os.environ:
    ipycompss.start(graph=True,
                    project_xml='../xml/project.xml',
                    resources_xml='../xml/resources.xml')
else:
    ipycompss.start(graph=True, monitor=1000)

********************************************************
**************** PyCOMPSs Interactive ******************
********************************************************
*          .-~~-.--.           ______         ______   *
*         :         )         |____  \       |____  \  *
*   .~ ~ -.\       /.- ~~ .      __) |          __) |  *
*   >       `.   .'       <     |__  |         |__  |  *
*  (         .- -.         )   ____) |   _    ____) |  *
*   `- -.-~  `- -'  ~-.- -'   |______/  |_|  |______/  *
*     (        :        )           _ _ .-:            *
*      ~--.    :    .--~        .-~  .-~  }            *
*          ~-.-^-.-~ \_      .~  .-~   .~              *
*                   \ \ '     \ '_ _ -~                *
*                    \`.\`.    //                      *
*           . - ~ ~-.__\`.\`-.//                       *
*       .-~   . - ~  }~ ~ ~-.~-.                       *
*     .' .-~      .-~       :/~-.~-./:                 *
*    /_~_ _ . - ~              

 

* - Log path : /home/javier/.COMPSs/Interactive_02/
* - PyCOMPSs Runtime started... Have fun!              *
********************************************************


In [6]:
from pycompss.api.task import task
from pycompss.api.parameter import FILE_IN

In [7]:
@task(returns=list, dataFile=FILE_IN)
def sortPartition(dataFile):
    '''
    Reads the dataFile and sorts its content which is assumed to be a dictionary {K: V}
    :param path: file that contains the data
    :return: a list of (K, V) pairs sorted.
    '''
    import pickle
    import operator
    with open(dataFile, 'rb') as f:
        data = pickle.load(f)
    # res = sorted(data, key=lambda (k, v): k, reverse=not ascending)
    partition_result = sorted(data.items(), key=operator.itemgetter(0), reverse=False)
    return partition_result

In [8]:
@task(returns=list, priority=True)
def reducetask(a, b):
    '''
    Merges two partial results (lists of (K, V) pairs) respecting the order
    :param a: Partial result a
    :param b: Partial result b
    :return: The merging result sorted
    '''
    partial_result = []
    i = 0
    j = 0
    while i < len(a) and j < len(b):
        if a[i] < b[j]:
            partial_result.append(a[i])
            i += 1
        else:
            partial_result.append(b[j])
            j += 1
    if i < len(a):
        partial_result + a[i:]
    elif j < len(b):
        partial_result + b[j:]
    return partial_result

In [9]:
def merge_reduce(function, data):
    import sys
    if sys.version_info[0] >= 3:
        import queue as Queue
    else:
        import Queue
    q = Queue.Queue()
    for i in data:
        q.put(i)
    while not q.empty():
        x = q.get()
        if not q.empty():
            y = q.get()
            q.put(function(x, y))
        else:
            return x

## MAIN

Parameters (that can be configured in the following cell):
* datasetPath: The path where the dataset is (default: the same as created previously).


In [10]:
import os
import time
from pycompss.api.api import compss_wait_on

datasetPath = directoryName  # Where the dataset is
files = []
for f in os.listdir(datasetPath):
    files.append(datasetPath + '/' + f)
    
startTime = time.time()

partialSorted = []
for f in files:
    partialSorted.append(sortPartition(f))
result = merge_reduce(reducetask, partialSorted)

result = compss_wait_on(result)

print("Elapsed Time(s)")
print(time.time() - startTime)
import pprint
pprint.pprint(result)

Found task: sortPartition
Found task: reducetask
Elapsed Time(s)
0.6505186557769775
[(0.07258938114096158, 172),
 (0.09625062293973508, 596),
 (0.12762366545414472, 653),
 (0.14556555856190778, 210),
 (0.22392646256577475, 510),
 (0.2769703043629682, 205),
 (0.3152663210413714, 15),
 (0.31828094294220544, 618),
 (0.3483232404597325, 499),
 (0.3712991143058685, 459),
 (0.37194548853221154, 450),
 (0.3968038855206313, 645),
 (0.48219488026361623, 858),
 (0.6322022686304333, 644),
 (0.6483470918550966, 654),
 (0.7261087234742485, 509),
 (0.7295318858073472, 547),
 (0.8656032998170026, 842),
 (0.9662456234342295, 671)]


In [11]:
ipycompss.stop()

********************************************************
***************** STOPPING PyCOMPSs ********************
********************************************************
Checking if any issue happened.
         have not been brought to the master.
********************************************************
